In [8]:
%matplotlib qt

import pandas as pd
import numpy as np
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import warnings
from sklearn.metrics import mean_squared_error
import os
import re
import glob
import seaborn as sns

#폴더에서 csv 전체를 읽고, 기준 데이터에서 피처를 선정

def read_csv_and_select_feature(folder_path, drop_col, str_col, del_if_zero):
    
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    if not csv_files:
        print ("csv 파일을 찾을 수 없습니다.")
    else:
        for file in csv_files:
            print(f"{os.path.basename(file)}")
        print(f"총{len(csv_files)}개의 파일을 찾았습니다\n")

    # 파일명을 통해 가장 최근 데이터를 선정하고 그 파일의 칼럼을 통합할 데이터의 기준으로
    most_recent_file = None
    max_year=0
    year_pattern = re.compile(r'\d{4}')
    
    for file in csv_files:
        file_name = os.path.basename(file)
        match = year_pattern.search(file_name)
        if match:
            current_year= int(match.group())
            if current_year > max_year:
                max_year = current_year
                most_recent_file= file

    try: 
        benchmark_col = pd.read_csv(most_recent_file, encoding="cp949", nrows=0)
    except UnicodeDecodeError:
        benchmark_col = pd.read_csv(most_recent_file, encoding="uft-8", nrows=0)
    print (f"가장 최근 조사인 {max_year}년 데이터에 {len(benchmark_col.columns)}개의 칼럼이 있습니다.\n")
    
    # drop_col에 있는 제외할 칼럼을 제거. 
    if drop_col: 
        all_col = benchmark_col.columns.tolist()
        filtered_col = [col for col in all_col if not any(re.search(keyword, col) for keyword in drop_col)]
        print(f"제외할 칼럼은 {drop_col}. {len(all_col)-len(filtered_col)}개의 칼럼이 제거되었습니다.\n{len(filtered_col)}개의 칼럼을 조사합니다.\n")
    else:
        filtered_col = benchmark_col.columns.tolist()
    #선택된 피쳐(열)가 있는 칼럼을 파일에서 읽은 후 데이타를 병합.
    df_list=[]
    warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
    
    for file in csv_files:
        
        try: 
            df_col=pd.read_csv(file, encoding="cp949", nrows=0).columns
            csv_encoding="cp949"
        except UnicodeDecodeError:
            df_col=pd.read_csv(file, encoding="utf-8", nrows=0).columns
            csv_encoding="uft-8"
    
        col_to_load= list(set(filtered_col) & set(df_col))
        if col_to_load != filtered_col:
            print(f"{os.path.basename(file)}에 {list(set(filtered_col) - set(df_col))}의 데이타가 없습니다.NaN으로 채웁니다.")
        df= pd.read_csv(file, encoding=csv_encoding, usecols=col_to_load)
        df=df.reindex(columns=filtered_col)
        
        df=df[1:]
        df_list.append(df)
    
    merged_df=pd.concat(df_list, ignore_index=True)
    print(f"\n총 데이터는 {len(merged_df.columns)}개의 열, {len(merged_df)}개의 행입니다.\n")

    #str_col은 문자 코드. 숫자 코드로 변환
    print(f"{str_col}은 숫자 코드로 변환합니다.\n")
    all_col=merged_df.columns.tolist()
    numberic_col = list(set(all_col) - set(str_col))
    for col in numberic_col:
        merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    le=LabelEncoder()
    for col in str_col:
        merged_df[col]=le.fit_transform(merged_df[col])
    
     #모델에 중요한 영향을 미치기 때문에 0이 되면 안 되는 피쳐 del_if_zero에서 값이 0인 샘플을 삭제
    print(f"{del_if_zero}에서 0이 될 수 없는 샘플을 삭제합니다.\n")
    len_before=len(merged_df)
    for col in del_if_zero:
        merged_df=merged_df[merged_df[col]>0]
    len_after =len(merged_df)
    print(f"{len_before - len_after}개의 샘플을 제거했습니다.\n")
    
    return merged_df


#_____________________________________________________________________
#0을 평균으로 대체, 0이 과반 이상이면 삭제

def zero_processing(df):
        
    df_temp=df.replace(0, np.nan)
    df_mean=df_temp.mean().round()
    df_semifinal=df_temp.fillna(df_mean)
    df_final=df_semifinal.fillna(0).astype(int)
    print("0을 평균값으로 대체. 열의 값이 모두 NaN인 경우 0으로 대체")
    
    #0이 과반 이상인 열을 삭제
    zeros_count = (df_final == 0).sum()
    columns_to_drop = zeros_count[zeros_count > (len(df_final) / 2)].index.tolist()
    print(f"\n0의 값이 절반을 초과하는 열: {columns_to_drop}을 삭제.\n")
    df_final = df_final.drop(columns=columns_to_drop)
    print(f"최종 {len(df_final.columns)+1}개의 피쳐를 조사합니다.")
    return df_final

#_____________________________________________________________________
#총근로시간, 총임금 계산

def total_hours_and_wage(df, keywords):

    df['상여금성과급총액']=df['상여금성과급총액']/12 #상여금성과급총액은 1년 단위로 집계. 월단위로 재계산.
    all_col=df.columns.tolist()
    
    for keyword in keywords:
        filtered_col = [col for col in all_col if re.search(keyword, col)]
        df[keyword]=df[filtered_col].sum(axis=1)
        df=df.drop(columns=filtered_col)
        print(f"\n<{keyword}> 칼럼 생성. {filtered_col} 칼럼 삭제.")

    return df

#_____________________________________________________________________

#XGboost 머신러닝

def XGboost_ML(df, target_col, weight_col, n_estimators=30):
    
    X = df.drop([target_col, weight_col], axis=1)
    y= df[target_col]
    w= df[weight_col]
    
    X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, w, test_size=0.3, random_state=42)
    
    model= xgb.XGBRegressor(objective="reg:squarederror", n_estimators=n_estimators, random_state=42)
    model.fit(X_train, y_train, sample_weight=w_train)
    
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, sample_weight=w_test))
    mean = y_test.mean()
    print("-" * 30)
    print(f"\nXGBoost 모델 성능 (RMSE): {rmse:.2f} \n 타깃 평균은 {mean:.2f}.\n 평균 대비 RMSE는 {rmse/mean:.2%}")
    print("-" * 30)
    print("\nXGBoost 모델의 SHAP 플롯을 생성합니다.\n")
    X_train_10= X_train.sample(frac=0.01, random_state=1)
    explainer = shap.Explainer(model, X_train_10)
    s_values= explainer(X_train_10)
    plt.rc('font', family='NanumGothic')
    shap.plots.bar(s_values, max_display=10)
   
    print("--- XGBoost 모델 분석 완료 ---")
    return s_values

#_____________________________________________________________________
#Deep-Learning 모델
def deep_learning(df):

    print("ANN 모델 분석을 시작합니다.", "*"*10)

    #훈련용, 검증용, 시험용 데이터 나누기
    X = df.drop([target_col, weight_col], axis=1)
    y= df[target_col]
    w= df[weight_col]
    X_train, X_temp, y_train, y_temp, w_train, w_temp = train_test_split(X, y, w, test_size=0.4, random_state=42)
    X_val, X_test, y_val,y_test, w_val, w_test =train_test_split(X_temp, y_temp, w_temp, test_size=0.5, random_state=36)

    # 피쳐 데이타 정규화
    mean = X_train.mean()
    X_train -=mean
    std= X_train.std()
    X_train /=std
    X_val -=mean
    X_val /=std
    X_test -= mean
    X_test /=std

    import keras
    from tensorflow.keras import layers
    model= keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(512, activation="relu"),
    layers.Dense(1)
    ])
    model.compile(optimizer=keras.optimizers.RMSprop(0.1), loss="mse", metrics=["mae"])
    history= model.fit(X_train, y_train, epochs=5, batch_size=512, sample_weight=w_train, validation_data=(X_val, y_val,w_val))
    
    print("분석을 끝냈습니다. 결과는 Keras의 History를 참조")

    return history

#_____________________________________________________________________
#데이터 기본 분석

def basic_analysis(df,target_col, weight_col):
    
    #Heat_Map 전처리
    from statsmodels.stats.weightstats import DescrStatsW
    weights =df[weight_col]
    df_data=df.drop(columns=weight_col)
    weighted_stats = DescrStatsW(df_data, weights=weights)
    weighted_corr_matrix = weighted_stats.corrcoef
    corr_df = pd.DataFrame(weighted_corr_matrix, columns=df_data.columns, index=df_data.columns)
    corr_df_s =corr_df.stack().reset_index(name='correlation')
    
    
    plt.rc('font', family='NanumGothic')
    plt.rcParams['axes.unicode_minus'] = False
    
    #히트맵
    g=sns.relplot(data=corr_df_s, x="level_0", y="level_1", hue="correlation", size="correlation", palette="vlag",hue_norm=(-1, 1), edgecolor=".7",
        height=10, sizes=(50, 250), size_norm=(-.2, .8),)
    g.set_xticklabels(rotation=90)
    plt.show()

    #타깃 값 히스토그램
    threshold= df[target_col].quantile(0.99)
    
    sns.displot(data=df,bins=1000, x=target_col, weights=weight_col, kde=True)
    plt.xlim(0,threshold)
    plt.show()

    #중요 변수 히스토그램
    
    max_list=corr_df.nlargest(5,target_col,keep='last').index
    classification_col=["코드","여부"]
    for col in max_list:
     if col != target_col:
         if any(keyword in col for keyword in classification_col): 
             sns.displot(data=df, x=target_col, weights=weight_col, hue=col, kind='kde')
             plt.xlim(0,threshold)
             plt.show()
         else:
             sns.displot(data=df, x=target_col, y=col)
             plt.xlim(0,threshold)
             plt.show()
    
#_____________________________________________________________________
    
def main():
    
        # 중요 파라미터들
    folder_path="data2" #csv 파일이 있는 폴더
    keywords=["근로시간수", "액"]
    
    
    drop_col = None #변수 설명하는 칼럼들
    str_col = ['산업대분류코드'] #분류코드가 String인 칼럼
    del_if_zero =["정액급여액"] #값이 0인 경우 삭제해야 하는 샘플
    #enterance_date_col="현재일관련사항_직장시작연월"
    #census_date_col ="조사연월"
    weight_col = "가중값"
    target_col="액"
    
    #함수 호출
    df= read_csv_and_select_feature(folder_path=folder_path, drop_col=drop_col, str_col=str_col, del_if_zero=del_if_zero)
    #df= zero_processing(df=df)
   # df= len_of_serv_calculator(df=df, enterance_date_col_name=enterance_date_col, census_date_col_name=census_date_col )
    df = total_hours_and_wage(df=df,keywords=keywords)
    basic_analysis(df, target_col=target_col, weight_col=weight_col)
    shap_values= XGboost_ML(df=df, target_col=target_col, weight_col=weight_col, n_estimators=100)


In [9]:
main()

2021_총괄_20250827_24235.csv
2022_총괄_20250827_24235.csv
2023_총괄_20250827_62500.csv
2024_총괄_20250827_62500.csv
총4개의 파일을 찾았습니다

가장 최근 조사인 2024년 데이터에 29개의 칼럼이 있습니다.

2021_총괄_20250827_24235.csv에 []의 데이타가 없습니다.NaN으로 채웁니다.
2022_총괄_20250827_24235.csv에 []의 데이타가 없습니다.NaN으로 채웁니다.
2023_총괄_20250827_62500.csv에 []의 데이타가 없습니다.NaN으로 채웁니다.
2024_총괄_20250827_62500.csv에 []의 데이타가 없습니다.NaN으로 채웁니다.

총 데이터는 29개의 열, 3959117개의 행입니다.

['산업대분류코드']은 숫자 코드로 변환합니다.

['정액급여액']에서 0이 될 수 없는 샘플을 삭제합니다.

0개의 샘플을 제거했습니다.


<근로시간수> 칼럼 생성. ['소정실근로시간수', '초과실근로시간수', '휴일실근로시간수'] 칼럼 삭제.

<액> 칼럼 생성. ['정액급여액', '초과급여액', '상여금성과급총액'] 칼럼 삭제.
------------------------------

XGBoost 모델 성능 (RMSE): 1844.59 
 타깃 평균은 4405.24.
 평균 대비 RMSE는 41.87%
------------------------------

XGBoost 모델의 SHAP 플롯을 생성합니다.



 99%|===================| 27523/27714 [02:47<00:01]        

--- XGBoost 모델 분석 완료 ---


In [ ]:
L